In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math
import copy

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

dm.Utilities.set_compute_backend('torch')

In [2]:
# WIP
# On va merge les deux .pkl en un seul dictionnary
data_source = pickle.load(open("../../data/leafbasi.pkl", 'rb'))
data_target = pickle.load(open("../../data/leafbasit.pkl", 'rb'))

Dx = 0.
Dy = 0.
height_source = 38.
height_target = 100.

source = torch.tensor(data_source[1]).type(torch.get_default_dtype())
target = torch.tensor(data_target[1]).type(torch.get_default_dtype())

smin, smax = torch.min(source[:, 1]), torch.max(source[:, 1])
sscale = height_source / (smax - smin)
source[:, 1] = Dy - sscale * (source[:, 1] - smax)
source[:, 0] = Dx + sscale * (source[:, 0] - torch.mean(source[:, 0]))

tmin, tmax = torch.min(target[:, 1]), torch.max(target[:, 1])
tscale = height_target / (tmax - tmin)
target[:, 1] = - tscale * (target[:, 1] - tmax)
target[:, 0] = tscale * (target[:, 0] - torch.mean(target[:, 0]))

pos_source = source[source[:, 2] == 2, 0:2]
pos_source = torch.tensor(np.delete(pos_source.numpy(), 3, axis=0))
pos_implicit0 = source[source[:, 2] == 1, 0:2]
pos_implicit1 = source[source[:, 2] == 1, 0:2]
pos_target = target[target[:, 2] == 2, 0:2]

aabb = dm.Utilities.AABB.build_from_points(pos_target)
aabb.squared()

In [3]:
# Some plots
%matplotlib qt5

plt.subplot(2, 2, 1)
plt.axis(aabb.totuple())
plt.title('Source')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')
plt.axis('equal')

plt.subplot(2, 2, 2)
plt.axis(aabb.totuple())
plt.title('Target')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')
plt.axis('equal')

plt.subplot(2, 2, 3)
plt.imshow(data_source[0])

plt.subplot(2, 2, 4)
plt.imshow(data_target[0])

plt.show()

In [4]:
# Setting up the modules

# Global translation module
global_translation = dm.DeformationModules.GlobalTranslation(2)

# Local translation module
sigma0 = 15.
nu0 = 0.001
coeff0 = 100.
implicit0 = dm.DeformationModules.ImplicitModule0(2, pos_implicit0.shape[0], sigma0, nu0, coeff0, gd=pos_implicit0.clone().requires_grad_())

# Elastic modules
sigma1 = 30.
nu1 = 0.001
coeff1 = 0.01
C = torch.zeros(pos_implicit1.shape[0], 2, 1)
K, L = 10, height_source
a, b = -2 / L ** 3, 3 / L ** 2
C[:, 1, 0] = (K * (a * (L - pos_implicit1[:, 1] + Dy) ** 3  + b * (L - pos_implicit1[:, 1] + Dy) ** 2))
C[:, 0, 0] = 1. * C[:, 1, 0]
th = 0. * math.pi * torch.ones(pos_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])

implicit1 = dm.DeformationModules.ImplicitModule1(2, pos_implicit1.shape[0], sigma1, C, nu=nu1, coeff=coeff1, gd=(pos_implicit1.clone().requires_grad_(), R.clone().requires_grad_()))

In [5]:
# Setting up the model and start the fitting loop
model = dm.Models.ModelPointsRegistration([pos_source.clone()], [global_translation, implicit0, implicit1], [dm.Attachment.VarifoldAttachment(2, [20., 60.], backend='torch')], lam=100.)
fitter = dm.Models.ModelFittingScipy(model)

In [6]:
shoot_method = 'euler'
shoot_it = 10

In [7]:
costs = fitter.fit([pos_target], 40, log_interval=1, options={'shoot_method': shoot_method, 'shoot_it': shoot_it})

Initial energy = 1813580.500


Time: 4.175846422091126
Iteration: 1 
Total energy = 1170181.375 
Attach cost = 1170137.25 
Deformation cost = 44.0951042175293


Time: 7.013536680024117
Iteration: 2 
Total energy = 906520.1875 
Attach cost = 906370.3125 
Deformation cost = 149.8910369873047


Time: 8.419079469982535
Iteration: 3 
Total energy = 349651.1875 
Attach cost = 349108.0 
Deformation cost = 543.180908203125


Time: 11.170904641970992
Iteration: 4 
Total energy = 60782.1484375 
Attach cost = 60062.890625 
Deformation cost = 719.2579956054688


...

Time: 68.91312305489555
Iteration: 36 
Total energy = 29427.439453125 
Attach cost = 28485.15625 
Deformation cost = 942.28271484375


Time: 70.47218667599373
Iteration: 37 
Total energy = 29424.97265625 
Attach cost = 28480.6640625 
Deformation cost = 944.3080444335938


Time: 71.97469987301156
Iteration: 38 
Total energy = 29423.185546875 
Attach cost = 28478.515625 
Deformation cost = 944.6697998046875


Time: 73.41382108000107
Iteration: 39 
Total energy = 29416.64453125 
Attach cost = 28470.3125 
Deformation cost = 946.332275390625


Time: 74.84998879302293
Iteration: 40 
Total energy = 29406.802734375 
Attach cost = 28458.59375 
Deformation cost = 948.2095947265625
Optimisation process exited with message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
Final energy = 29406.802734375
Closure evaluations = 52
Time elapsed = 74.85131214605644


In [8]:
# Results
intermediate_states, _ = model.compute_deformed(shoot_method, shoot_it, intermediates=True)

deformed_source = intermediate_states[-1][0].gd
deformed_implicit0 = intermediate_states[-1][2].gd
deformed_implicit1 = intermediate_states[-1][3].gd[0]

%matplotlib qt5
plt.subplot(1, 3, 1)
plt.title("Source")
plt.axis(aabb.totuple())
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')
plt.axis('equal')

plt.subplot(1, 3, 2)
plt.title("Deformed source")
plt.axis(aabb.totuple())
plt.plot(deformed_source[:, 0], deformed_source[:, 1], '-')
plt.plot(deformed_implicit0[:, 0], deformed_implicit0[:, 1], 'x')
plt.plot(deformed_implicit1[:, 0], deformed_implicit1[:, 1], '.')
plt.axis('equal')

plt.subplot(1, 3, 3)
plt.title("Deformed source and target")
plt.axis(aabb.totuple())
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')
plt.plot(deformed_source[:, 0], deformed_source[:, 1], '-')
plt.axis('equal')
plt.show()

In [142]:
modules = dm.DeformationModules.CompoundModule(copy.copy(model.modules))
modules.manifold.fill(model.init_manifold.clone(), copy=True)

intermediate_states, intermediate_controls = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(modules), shoot_it, shoot_method, intermediates=True)

implicit0_controls = [control[2] for control in intermediate_controls]
implicit1_controls = [control[3] for control in intermediate_controls]

print(implicit1_controls)

[tensor([-0.0180]), tensor([0.0365]), tensor([0.0865]), tensor([0.1399]), tensor([0.2030]), tensor([0.2786]), tensor([0.3508]), tensor([0.3607]), tensor([0.3003]), tensor([0.2561])]


In [149]:
modules = dm.DeformationModules.CompoundModule(copy.copy(model.modules))
modules.manifold.fill(model.init_manifold.clone(), copy=True)

silent = copy.copy(modules[0])
deformation_grid = dm.DeformationModules.DeformationGrid(dm.Utilities.AABB.build_from_points(pos_source).scale(1.2), [32, 32])
implicit1 = copy.copy(modules[3])

# silent.manifold.fill(model.init_manifold[0], copy=True)
# silent.manifold.fill_cotan_zeros()
# implicit0.manifold.fill(model.init_manifold[3], copy=True)
# implicit1.manifold.fill_cotan_zeros()

controls = [[torch.tensor([]), torch.tensor([]), control] for control in implicit1_controls]

dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, deformation_grid, implicit1]), shoot_it, shoot_method, controls=controls)

deformed_source = silent.manifold.gd.detach()
deformed_grid = deformation_grid.togrid()
deformed_implicit1 = implicit1.manifold.gd[0].detach()

ax = plt.subplot()
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '--', color='black')
plt.plot(deformed_source[:, 0].numpy(), deformed_source[:, 1].numpy(), '-', color='black')
dm.Utilities.plot_grid(ax, deformed_grid[0], deformed_grid[1], color='xkcd:light blue', lw=0.5)
plt.plot(deformed_implicit1[:, 0].numpy(), deformed_implicit1[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()

In [155]:
modules = dm.DeformationModules.CompoundModule(copy.copy(model.modules))
modules.manifold.fill(model.init_manifold.clone(), copy=True)

silent = copy.copy(modules[0])
deformation_grid = dm.DeformationModules.DeformationGrid(dm.Utilities.AABB.build_from_points(pos_source).scale(1.2), [32, 32])
implicit0 = copy.copy(modules[2])

controls = [[torch.tensor([]), torch.tensor([]), control] for control in implicit0_controls]

dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, deformation_grid, implicit0]), shoot_it, shoot_method, controls=controls)

deformed_source = silent.manifold.gd.detach()
deformed_grid = deformation_grid.togrid()
deformed_implicit0 = implicit0.manifold.gd.detach()

ax = plt.subplot()
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '--', color='black')
plt.plot(deformed_source[:, 0].numpy(), deformed_source[:, 1].numpy(), '-', color='black')
dm.Utilities.plot_grid(ax, deformed_grid[0], deformed_grid[1], color='xkcd:light blue', lw=0.5)
plt.plot(deformed_implicit0[:, 0].numpy(), deformed_implicit0[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()